In [1]:
# !pip install geemap

# EO4BEE

**This webapp provide a tool to visualize various earth observation data to support the decision-making process of beekeeping in Europe**

1. Select the earth observation variable that you're interested in.
2. Drag the time slider to visualize the image at a specific time which you want to inspect. 
3. Click on the tool icon and then the plot icon to plot the change of the chosen variable over time at a specific location.
4. If you want to visualize the change smoothly as a gif, use the create timelapse tool to create and download customized timelapse gif.

In [2]:
import os
import ee
import geemap
import ipywidgets as widgets

In [3]:
style = {'description_width': 'initial'}
layer = widgets.Dropdown(
    description='Select Layer:',
    options=['Annual Average Temperature','Monthly Average Temperature','Annual Total Precipitation','Monthly Total Precipitation','NDVI','Landcover','Soil Moisture', 
             'PM2.5','Sulphur Dioxide'],
#     options=['Annual Average Temperature','Monthly Average Temperature'],
    value='Annual Average Temperature',
    style=style
)
show_layer = widgets.Button(
    description='Visualize',
    button_style='primary',
    tooltip='Click the add the selected layer to the map',
    style=style
)
hbox = widgets.HBox([layer, show_layer])
hbox

In [4]:
# Map.remove_control(Map.slider_ctrl)
# Map.add_control(Map.slider_ctrl)
# Map.slider_ctrl.widget.children[4].click()

In [5]:
Map = geemap.Map()
Map.set_center(14, 52, 4)
Map

Map(center=[52, 14], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [6]:
def toCelsius(image):
    celsius = image.subtract(273.15) 
    return celsius

In [7]:
def showLayer():
    if layer.value == 'Annual Average Temperature':
        era5_complete = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('mean_2m_air_temperature')
        distinctYear = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('mean_2m_air_temperature').distinct('year')
        filter = ee.Filter.equals(leftField='year', rightField= 'year')
        join = ee.Join.saveAll('sameYear')
        joinCol = ee.ImageCollection(
          join.apply(distinctYear, era5_complete, filter)
        )

        def func_lgr(img):
          yearCol = ee.ImageCollection.fromImages(img.get('sameYear'))
          return yearCol.mean().set('Year', img.get('year'))

        annualMeanCol = joinCol.map(func_lgr)
        annualMeanCol_celsius = annualMeanCol.map(toCelsius)
        image = annualMeanCol_celsius.toBands()
        vis_params = {
            'min': -23.15,
            'max': 46.85,
            'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
            ]
        }
    #     Map.addLayer(image, {}, "Time series", False)
        Map.add_time_slider(image, vis_params, '','Time series of annual average air temperature(℃) at 2m', time_interval=1)
    
    if layer.value == 'Monthly Average Temperature':
        collection = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('mean_2m_air_temperature')
        vis_params = {
            'min': -23.15,
            'max': 46.85,
            'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
            ]
        }
        era5_celsius = collection.map(toCelsius)
        image = era5_celsius.toBands()
        Map.add_time_slider(image, vis_params, '','Time series of monthly average air temperature(℃) at 2m', time_interval=1)
    
    if layer.value == 'Annual Total Precipitation':
        era5_complete = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation')
        distinctYear = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation').distinct('year')
        filter = ee.Filter.equals(leftField='year', rightField= 'year')
        join = ee.Join.saveAll('sameYear')
        joinCol = ee.ImageCollection(
          join.apply(distinctYear, era5_complete, filter)
        )

        def func_lgr(img):
          yearCol = ee.ImageCollection.fromImages(img.get('sameYear'))
          return yearCol.sum().set('Year', img.get('year'))

        annualSumCol = joinCol.map(func_lgr)
        image = annualSumCol.toBands()
        vis_params = {
              'min': 0,
              'max': 4.8,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
    #     Map.addLayer(image, {}, "Time series", False)
        Map.add_time_slider(image, vis_params, '','Time series of annual average air temperature(℃) at 2m', time_interval=1)
    
    if layer.value == 'Monthly Total Precipitation':
        collection  = ee.ImageCollection('ECMWF/ERA5/MONTHLY').select('total_precipitation')
        vis_params = {
              'min': 0,
              'max': 0.4,
              'palette': ['#f7fcf0','#e0f3db','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#0868ac','#084081']
            }
        image = collection.toBands()
        Map.add_time_slider(image, vis_params, '','Time series of monthly total precipitation(m)', time_interval=1)

In [8]:
def getLayerName(layer_now):
    if layer_now == 'Monthly Average Temperature':
        return 'Time series of monthly average air temperature(℃) at 2m'
    elif layer_now == "Annual Average Temperature":
        return 'Time series of annual average air temperature(℃) at 2m'
    else:
        return layer_now

def add_clicked(b):
#     remove current layers and slider control
    if hasattr(Map,'slider_ctrl'):
        Map.slider_ctrl.widget.children[4].click()
    showLayer()
    
show_layer.on_click(add_clicked)